In [10]:
array_one = ["¤", "U", "¢ð", "´ð", "ð¸", "Ò", "¥æò", "¸", "¸•", "¸¹", "¸»", "¸Á", "¸Ç", "¸É", "¸È", "¸Ø", "¸Ú", "¸Ù", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "T", "V", "W", "X", "Y", "`", "a", "b", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "p", "q", "r", "s", "t", "u", "žæ", "ž", "#", "%", "@", "„", "¦", "¨", "¯", "µ", "º", "Cþ", "q", "Ê", "u", "g", "Ÿæ", "Åþ", "Çþ", "Éþ", "^", "h", "Ð", "ý", "þ", "¥ô", "¥æð", "¥õ", "¥æñ", "¥æ", "¥", "§Z", "§ü", "§", "©", "ª", "«", "¬", "­", "°ð", "°", "€", "·", "", "¹", "‚", "»", "ƒæ", "ƒ", "¾", "“", "‘", "¿", "À", "”", "…", "’", "Á", "Ûæ", "Û", "†æ", "†", "Å", "Æ", "Ç", "É", "‡æ", "‡", "ˆ", "Ì", "‰", "Í", "Î", "¼", "Š", "Ï", "óæ", "ó", "‹æ", "Ù", "‹", "Œ", "Â", "", "È", "Ž", "Õ", "", "Ö", "", "×", "Ä", "Ø", "Ú", "Ë", "Ü", "¶", "Ý", "Ã", "ß", "àæ", "³æ", "o", "³", "à", "c", "á", "S", "â", "ã", "±", "ÿæ", "ÿ", "˜æ", "˜", "™æ", "™", "üð´", "æò", "æñ", "æ", "è", "é", "ê", "ä", "å", "ë", "ì", "í", "Ô", "ñ", "ô", "õ", "¢", "´", "¡", "Ñ", "¸", "ò", "ù", "÷", "ð", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "®", "v", "w", "x", "y", "z", "{", "|", "}", "~", "्ो", "्ौ", "्ाे", "्ाा", "ाे", "ाे", "ाै", "्ा", "ंु", "ओे", "ोे", "ाे", "ईंं"]
array_two = ["", "", "ð¢", "ð´", "¸ð", "'", "ऑ", "फ़्", "क़", "ख़", "ग़", "ज़", "ड़", "ढ़", "फ़", "य़", "ऱ", "ऩ", "्र", "क्च", "ष्ट", "ष्ठ", "श्व", "स्न", "त्र", "॥", "ढ्ढ", "छ्व", "्य", "रु", "रू", "हृ", "ह्र", "क्क", "क्त", "क्र", "ञ्ज", "ङ्क", "ङ्ख", "ङ्ग", "ङ्घ", "क्व", "ड्ड", "ड्ढ", "स्र", "द्ग", "द्घ", "द्द", "द्ध", "द्ब", "द्भ", "द्म", "द्य", "द्व", "ठ्ठ", "श्च", "ह्न", "ह्म्", "ह्य", "ह्ल", "ह्व", "त्त", "त्त्", "प्त", "त्न", "ञ्च", "ल्ल", "ष्ट्व", "ङ्क्ष", "ख्न", "द्ब्र", "ख्र", "ष्ट्र", "ह्न", "ज़्", "ह्व", "द्द", "श्र", "ट्र", "ड्र", "ढ्र", "ट्ट", "द्ध", "।", "्र", "्र", "ओ", "ओ", "औ", "औ", "आ", "अ", "ईं", "ई", "इ", "उ", "ऊ", "ऋ", "ॠ", "ऌ", "ऐ", "ए", "क्", "क", "ख्", "ख", "ग्", "ग", "घ", "घ्", "ङ", "च्च्", "च्", "च", "छ", "ज्ज्", "ज्", "ज्", "ज", "झ", "झ्", "ञ", "ञ्", "ट", "ठ", "ड", "ढ", "ण", "ण्", "त्", "त", "थ्", "थ", "द", "द", "ध्", "ध", "न्न", "न्न्", "न", "न", "न्", "प्", "प", "फ्", "फ", "ब्", "ब", "भ्", "भ", "म्", "म", "य्", "य", "र", "ल्", "ल", "ल", "ळ", "व्", "व", "श", "श", "श", "श्", "श्", "ष्", "ष", "स्", "स", "ह", "ह्", "क्ष", "क्ष्", "त्र", "त्र्", "ज्ञ", "ज्ञ्", "ðZ", "ॉ", "ौ", "ा", "ी", "ु", "ू", "ु", "ू", "ृ", "ॄ", "ॢ", "े", "ै", "ो", "ौ", "ं", "ं", "ँ", ":", "़", "ॅ", "ऽ", "्", "े", "०", "१", "२", "३", "४", "५", "६", "७", "८", "९", "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "े", "ै", "े", "ा", "ो", "ो", "ौ", "", "ुं", "ओ", "ो", "ो", "ईं"]

In [11]:
def chanakya_unicode_convertor(input_text):

    # Check if input text is not empty
    # Replace specific characters with their Unicode equivalents
    input_text = input_text.replace("क़", "क़")
    input_text = input_text.replace("ख़‌", "ख़")
    input_text = input_text.replace("ग़", "ग़")
    input_text = input_text.replace("ज़", "ज़")
    input_text = input_text.replace("ड़", "ड़")
    input_text = input_text.replace("ढ़", "ढ़")
    input_text = input_text.replace("ऩ", "ऩ")
    input_text = input_text.replace("फ़", "फ़")
    input_text = input_text.replace("य़", "य़")
    input_text = input_text.replace("ऱ", "ऱ")

    # Perform general replacements using predefined arrays
    for i in range(len(array_one)):
        while array_one[i] in input_text:
            input_text = input_text.replace(array_one[i], array_two[i])

    # Replace special character 'Z' with 'üं'
    input_text = input_text.replace("Z", "üं")

    # Handle special case of character 'ç'
    position_of_f = input_text.find("ç")
    while position_of_f != -1:
        character_right_to_f = input_text[position_of_f + 1]
        input_text = input_text.replace("ç" + character_right_to_f, character_right_to_f + "ि")
        position_of_f += 1
        # Handle additional replacements after 'ç'
        while position_of_f < len(input_text) - 1 and input_text[position_of_f + 1] == "्":
            string_to_be_replaced = input_text[position_of_f + 1: position_of_f + 3]
            input_text = input_text.replace("ि" + string_to_be_replaced, string_to_be_replaced + "ि")
            position_of_f += 2
        position_of_f = input_text.find("ç", position_of_f + 1)

    # Define set of matras for further processing
    set_of_matras = "ा ि ी ु ू ृ े ै ो ौ ं ः ँ ॅ"

    # Handle special case of character 'ü'
    position_of_z = input_text.find("ü")
    while position_of_z > 0:
        probable_position_of_half_r = position_of_z - 1
        character_at_probable_position_of_half_r = input_text[probable_position_of_half_r]
        # Find the character before 'ü' which is not a matra
        while character_at_probable_position_of_half_r in set_of_matras:
            probable_position_of_half_r -= 1
            character_at_probable_position_of_half_r = input_text[probable_position_of_half_r]
        # Replace characters around 'ü' with 'र्'
        character_to_be_replaced = input_text[probable_position_of_half_r:position_of_z]
        new_replacement_string = "र्" + character_to_be_replaced
        character_to_be_replaced += "ü"
        input_text = input_text.replace(character_to_be_replaced, new_replacement_string)
        position_of_z = input_text.find("ü")

    # Return the modified text
    return input_text


In [5]:
# Example usage:
input_string = "ãðU ÂýÖé, ãðU ßâéÎðß-Âé ̃æ Ÿæè·ë¤c‡æ, ãðU âßüÃØæÂè Ö»ßæÙ÷, ×ñ ́ ¥æÂ·¤ô âæÎÚU Ù×S·¤æÚU ·¤ÚUÌæ ãê¡UÐ "
modified_string = chanakya_unicode_convertor(input_string)
print(modified_string)

हे प्रभु, हे वसुदेव-पु ̃ा श्रीकृष्ण, हे सर्वव्यापी भगवान्, मै ́ आपको सादर नमस्कार करता हूँ। 


In [6]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 35.1 MB/s eta 0:00:00


In [14]:
import fitz  # PyMuPDF

file_path = 'canto1_ch1.pdf'

pdf_document = fitz.open(file_path)

all_data = []

def convert_to_hindi(text, font_name):
  if "chanakya" in font_name.lower():
    return chanakya_unicode_convertor(text)
  else:
    return text

def get_font_style(font_name):
  if "bold" in font_name.lower():
    return "bold"
  elif "italic" in font_name.lower():
    return "italic"
  else:
    return "regular"

for page_num in range(len(pdf_document)):
  page_data = []
  page = pdf_document.load_page(page_num)
  text_instances = page.get_text("dict")  # Get text as a list of dictionaries

  for block in text_instances["blocks"]:
    for line in block["lines"]:
      for span in line["spans"]:
        # Keep only "size", "font", and "text" keys
        span_data = {
            "size": int(span["size"]),  # Convert "size" to integer
            "fontStyle": get_font_style(span["font"]),
            "color": span["color"] > 0,
            "text": convert_to_hindi(span["text"],span["font"]),
        }
        page_data.append(span_data)

  page_data = page_data[:-3]
  all_data.extend(page_data)

print(len(all_data))
pdf_document.close()


2069


In [78]:
# Initialize chunked_data
chunked_data = []

chunk = []

for i, entry in enumerate(all_data):
  font_size = entry["size"]
  if i == len(all_data) - 1:
    chunked_data.append(chunk)
    chunk = []
  elif font_size != 11 and font_size != 15:
    chunked_data.append(chunk)
    chunk = []
  elif font_size == 11 and (all_data[i + 1]["size"] != 11 or all_data[i - 1]["size"] != 11):
    chunked_data.append(chunk)
    chunk = []
  elif font_size == 15 and (all_data[i + 1]["size"] != 15 or all_data[i - 1]["size"] != 15):
    chunked_data.append(chunk)
    chunk = []
  elif "तात्पर्य" == entry["text"]:
    chunked_data.append(chunk)
    chunk = []

  chunk.append(entry)


# Merge the first four chunks into one array
merged_chunk = []
for i in range(4):
    merged_chunk.extend(chunked_data[i])

# Update chunked_data with the merged chunk
chunked_data = [merged_chunk] + chunked_data[4:]

print(len(chunked_data))

193


In [90]:
# Initialize verses_data
verses_data = []
verse = []

verses_data.append([chunked_data[0]])
for chunk in chunked_data[1:]:
  breakIntoVerse = False
  for entry in chunk:
    if "तात्पर्य" == entry["text"]:
      breakIntoVerse = True

  verse.append(chunk)

  if breakIntoVerse:
    verses_data.append(verse)
    verse = []

print(len(verses_data))

24


In [92]:
# Create HTML content
html_content = "<!DOCTYPE html>\n<html>\n<head>\n"
html_content += '<link rel="stylesheet" type="text/css" href="styles.css">\n'
html_content += "</head>\n<body>\n<p>"

for verse in verses_data:
  html_content += "<hr>"
  for chunk in verse:
    for entry in chunk:
        font_size = entry["size"]
        text = entry["text"]
        is_colored = entry.get("color", 0)  # Default to 0 if "color" key doesn't exist
        color = f'rgb(0, 0, {255 if is_colored else 0})'
        font_name = entry["fontStyle"]

        if "bold" in font_name.lower():
            html_content += f'<span style="font-weight: bold; font-size: {font_size}px; color: {color};">{text}</span>'
        elif "italic" in font_name.lower():
            html_content += f'<span style="font-style: italic; font-size: {font_size}px; color: {color};">{text}</span>'
        else:
            html_content += f'<span style="font-size: {font_size}px; color: {color};">{text}</span>'
    html_content += "<br>"

html_content += "</p>\n</body>\n</html>"

# Write HTML content to file
with open("output.html", "w") as html_file:
    html_file.write(html_content)

print("HTML generated successfully!")


HTML generated successfully!
